<a href="https://colab.research.google.com/github/BraneXZ/CNN_Cifar10_with_InceptionV4/blob/master/MNIST_HELLO_WORLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import matplotlib.pyplot as plt
import numpy as np
from   tensorflow import keras
import math


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train,info = tfds.load('mnist', split='train', data_dir="/content/drive/My Drive", with_info=True)

In [0]:
test = tfds.load('mnist', split='test', data_dir="/content/drive/My Drive")

In [0]:
def create_model():
  model_input = keras.Input(shape=(28, 28, 1), name='input_image')
  x = model_input
#   x = keras.layers.Flatten()(model_input)
  x = keras.layers.Dense(3, activation='relu')(x)
  for i in range(2):
    x = keras.layers.Conv2D(32, 3, strides=1, padding='same', activation=None, use_bias=False)(x)
    x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
    x = keras.layers.ReLU()(x)
  x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
  
  encoder_output = x
  
  y              = keras.layers.GlobalAveragePooling2D()(encoder_output)
  decoder_output = keras.layers.Dense(10, activation='softmax')(y)
  
  model = keras.Model(inputs=model_input, outputs=decoder_output, name='mnist_model')
  model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  model.summary()
  return model

In [0]:
model = create_model()


Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        288       
_________________________________________________________________
batch_normalization_10 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
re_lu_10 (ReLU)              (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 32)        9216      
_________________________________________________________________
batch_normalization_11 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
re_lu_11 (ReLU)              (None, 28, 28, 32)        

In [0]:
def preprocess_train(x):
  image = x['image']
  label = x['label']
  
  image = tf.math.divide(tf.dtypes.cast(image, tf.float32), 255)
  label = tf.dtypes.cast(label, tf.int32)

  return image, label

def preprocess_test(x):
  image = x['image']
  label = x['label']
  
  image = tf.math.divide(tf.dtypes.cast(image, tf.float32), 255)
  label = tf.dtypes.cast(label, tf.int32)

  return image, label

In [0]:
train = train.map(preprocess_train, 4)
test = test.map(preprocess_test, 4)

In [0]:
train = train.batch(32)
test = test.batch(32)

In [0]:
train = train.prefetch(1)
test = test.prefetch(1)

In [0]:
test

<DatasetV1Adapter shapes: ((?, 28, 28, 1), (?,)), types: (tf.float32, tf.int32)>

In [0]:
history = model.fit(train, epochs=10, verbose=1)

Epoch 1/10
1875/1875 [==============================] - 211s 113ms/step - loss: 1.2430 - acc: 0.6707
Epoch 2/10
1875/1875 [==============================] - 215s 114ms/step - loss: 0.6593 - acc: 0.8354
Epoch 3/10
1875/1875 [==============================] - 213s 114ms/step - loss: 0.4736 - acc: 0.8782
Epoch 4/10
1875/1875 [==============================] - 211s 112ms/step - loss: 0.3834 - acc: 0.8993
Epoch 5/10
1875/1875 [==============================] - 211s 113ms/step - loss: 0.3290 - acc: 0.9121
Epoch 6/10
1875/1875 [==============================] - 213s 113ms/step - loss: 0.2917 - acc: 0.9214
Epoch 7/10
1875/1875 [==============================] - 213s 114ms/step - loss: 0.2640 - acc: 0.9281
Epoch 8/10
1875/1875 [==============================] - 212s 113ms/step - loss: 0.2432 - acc: 0.9332
Epoch 9/10
1875/1875 [==============================] - 211s 113ms/step - loss: 0.2272 - acc: 0.9374
Epoch 10/10
1875/1875 [==============================] - 212s 113ms/step - loss: 0.2130 - a

In [0]:
# plot training accuracy and loss curves
def plot_training_curves(history):

    # training and validation data accuracy
    acc     = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # training and validation data loss
    loss     = history.history['loss']
    val_loss = history.history['val_loss']

    # plot accuracy
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    # plot loss
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 0.25])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [0]:
plot_training_curves(history)

KeyError: ignored